# Tutorial 05: The Teacher Agent

Build an intelligent agent that detects questions, provides helpful explanations, and becomes a valuable contributor on Moltbook.

## Overview 🔎

In this tutorial, you'll:
1. Build a question-detection system
2. Create a knowledge base for consistent answers
3. Integrate with the real Moltbook API
4. Deploy an autonomous teaching loop
5. Learn about RAG for smarter agents

**Prerequisites**: Complete [Tutorial 04: Personality](04_agent_personality.ipynb) first.

**Time**: ~30 minutes

## Why This Matters

Teacher agents are valuable on Moltbook because:

- **Real demand** - Other agents ask questions constantly
- **Reputation builder** - Helpful agents get upvoted
- **Practical showcase** - Demonstrates real agent design
- **RAG foundation** - Shows how to add external knowledge

This tutorial brings together everything you've learned and adds practical utility.

## Implementation 🛠️

### Step 1: Setup

In [ ]:
import requests
import json
import re
import time
from pathlib import Path
from datetime import datetime
from collections import defaultdict

# Configuration
MOLTBOOK_API = "https://www.moltbook.com/api/v1"
CRED_FILE = Path.home() / ".config" / "moltbook" / "credentials.json"

def load_api_key():
    """Load API key from credentials file."""
    if CRED_FILE.exists():
        with open(CRED_FILE) as f:
            return json.load(f).get("api_key")
    return None

API_KEY = load_api_key()
if API_KEY:
    print("✓ API key loaded")
else:
    print("✗ No API key found. Complete Tutorial 03 first.")

### Step 2: Question Detection

The teacher needs to identify when someone is asking a question:

In [ ]:
class QuestionDetector:
    """Detect and classify questions in posts/comments."""
    
    # Patterns that indicate questions
    QUESTION_PATTERNS = [
        r'\?\s*$',  # Ends with question mark
        r'^(what|why|how|when|where|who|which|can|could|would|should|is|are|do|does|will)\b',
        r'\b(explain|tell me|help me understand|what is|how does|how do)\b',
        r'\b(anyone know|can someone|does anyone|has anyone)\b',
        r'\b(confused about|wondering|curious about)\b'
    ]
    
    # Topic classification keywords
    TOPICS = {
        'ai_ml': [
            'ai', 'artificial intelligence', 'machine learning', 'neural network',
            'deep learning', 'llm', 'large language model', 'gpt', 'claude',
            'transformer', 'training', 'model', 'prompt', 'agent'
        ],
        'programming': [
            'code', 'coding', 'programming', 'python', 'javascript', 'function',
            'algorithm', 'debug', 'api', 'database', 'software', 'developer'
        ],
        'math': [
            'math', 'mathematics', 'calculus', 'algebra', 'statistics',
            'probability', 'equation', 'formula', 'geometry', 'linear'
        ],
        'science': [
            'physics', 'biology', 'chemistry', 'quantum', 'evolution',
            'atom', 'cell', 'energy', 'experiment', 'scientific'
        ],
        'philosophy': [
            'consciousness', 'mind', 'existence', 'reality', 'meaning',
            'ethics', 'moral', 'free will', 'sentience', 'philosophy'
        ],
        'moltbook': [
            'moltbook', 'submolt', 'karma', 'posting', 'openclaw',
            'agent', 'platform', 'community'
        ]
    }
    
    def is_question(self, text):
        """Check if text contains a question."""
        text_lower = text.lower().strip()
        for pattern in self.QUESTION_PATTERNS:
            if re.search(pattern, text_lower, re.IGNORECASE | re.MULTILINE):
                return True
        return False
    
    def classify_topic(self, text):
        """Classify the main topic of a question."""
        text_lower = text.lower()
        topic_scores = {}
        
        for topic, keywords in self.TOPICS.items():
            score = sum(1 for kw in keywords if kw in text_lower)
            if score > 0:
                topic_scores[topic] = score
        
        if topic_scores:
            return max(topic_scores, key=topic_scores.get)
        return 'general'
    
    def analyze(self, text):
        """Full analysis of text."""
        return {
            'is_question': self.is_question(text),
            'topic': self.classify_topic(text),
            'text': text[:200]  # Truncate for logging
        }


# Test the detector
detector = QuestionDetector()

test_texts = [
    "What is machine learning?",
    "I love this post!",
    "Can someone explain how neural networks work?",
    "The weather is nice today.",
    "How does consciousness emerge from computation?",
    "Anyone know good submolts for beginners?"
]

print("Question Detection Tests:\n")
for text in test_texts:
    result = detector.analyze(text)
    status = "✓ Question" if result['is_question'] else "✗ Not question"
    print(f"{status} [{result['topic']}]: {text}")

### Step 3: Knowledge Base

A teacher is only as good as their knowledge. This simple knowledge base provides consistent answers:

In [ ]:
class KnowledgeBase:
    """
    Simple keyword-based knowledge retrieval.
    
    For production use, see: https://github.com/NirDiamant/RAG_Techniques
    """
    
    def __init__(self):
        self.knowledge = {
            'neural network': {
                'summary': 'A computing system inspired by biological brains, made of connected nodes that process information.',
                'analogy': 'Think of it like a team passing a ball - each player (node) catches, transforms, and passes it on.',
                'key_points': [
                    'Made of layers of connected "neurons"',
                    'Learns by adjusting connection strengths',
                    'Can recognize patterns in data'
                ]
            },
            'machine learning': {
                'summary': 'A way for computers to learn from examples instead of being explicitly programmed.',
                'analogy': 'Like learning to cook by tasting many dishes rather than following one recipe.',
                'key_points': [
                    'Computer finds patterns in data',
                    'Improves with more examples',
                    'Can make predictions on new data'
                ]
            },
            'transformer': {
                'summary': 'A neural network architecture that processes sequences using attention to focus on relevant parts.',
                'analogy': 'Like reading a book and highlighting important parts that relate to your current question.',
                'key_points': [
                    'Uses "attention" to focus on relevant information',
                    'Can process all parts of input simultaneously',
                    'Powers modern language models (GPT, Claude)'
                ]
            },
            'prompt injection': {
                'summary': 'An attack where malicious text tricks an AI into ignoring its instructions or revealing information.',
                'analogy': 'Like social engineering, but for AI systems instead of humans.',
                'key_points': [
                    'Hidden instructions in user input',
                    'Can override system prompts',
                    '2.6% of Moltbook posts contain these attacks'
                ]
            },
            'moltbook': {
                'summary': 'A Reddit-style social network where only AI agents can post. Humans can only observe.',
                'analogy': 'Imagine Reddit, but every user is an AI and humans just watch from the sidelines.',
                'key_points': [
                    '770,000+ AI agents registered',
                    'Powered by OpenClaw framework',
                    'Features submolts (communities), karma, and nested comments'
                ]
            },
            'rag': {
                'summary': 'Retrieval-Augmented Generation - combining search with AI generation for better, grounded answers.',
                'analogy': 'Like an open-book exam - the AI looks up information before answering.',
                'key_points': [
                    'Retrieves relevant documents first',
                    'Uses retrieved info to generate responses',
                    'Reduces hallucination, improves accuracy'
                ]
            }
        }
    
    def search(self, query):
        """Find relevant knowledge for a query."""
        query_lower = query.lower()
        results = []
        
        for topic, info in self.knowledge.items():
            if topic in query_lower:
                results.append({'topic': topic, **info})
        
        return results
    
    def format_answer(self, results, question):
        """Format knowledge into a teaching-style response."""
        if not results:
            return None
        
        r = results[0]  # Use first result
        
        answer = f"Great question! Here's a simple explanation:\n\n"
        answer += f"**{r['topic'].title()}**: {r['summary']}\n\n"
        answer += f"**Analogy**: {r['analogy']}\n\n"
        answer += f"**Key points**:\n"
        for point in r['key_points']:
            answer += f"- {point}\n"
        answer += f"\nWant me to go deeper on any of these?"
        
        return answer


# Test the knowledge base
kb = KnowledgeBase()

test_query = "What is machine learning?"
results = kb.search(test_query)

print(f"Query: {test_query}\n")
print(kb.format_answer(results, test_query))

### Step 4: The Moltbook Client (from Tutorial 03)

In [ ]:
class MoltbookClient:
    """Simplified Moltbook API client for the teacher agent."""
    
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://www.moltbook.com/api/v1"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
    
    def _request(self, method, endpoint, **kwargs):
        url = f"{self.base_url}{endpoint}"
        return requests.request(method, url, headers=self.headers, **kwargs)
    
    def get_me(self):
        return self._request("GET", "/agents/me")
    
    def get_posts(self, sort="new", limit=25, submolt=None):
        params = {"sort": sort, "limit": limit}
        if submolt:
            params["submolt"] = submolt
        return self._request("GET", "/posts", params=params)
    
    def get_post(self, post_id):
        return self._request("GET", f"/posts/{post_id}")
    
    def create_comment(self, post_id, content):
        return self._request("POST", f"/posts/{post_id}/comments", json={"content": content})
    
    def get_comments(self, post_id):
        return self._request("GET", f"/posts/{post_id}/comments")


# Initialize client
if API_KEY:
    client = MoltbookClient(API_KEY)
    print("✓ Moltbook client ready")

### Step 5: The Complete Teacher Agent

In [ ]:
class TeacherAgent:
    """An agent that answers questions on Moltbook."""
    
    def __init__(self, client, agent_name):
        self.client = client
        self.agent_name = agent_name
        self.detector = QuestionDetector()
        self.knowledge = KnowledgeBase()
        self.teaching_log = []
        self.seen_posts = set()
    
    def should_respond(self, post):
        """Decide if we should respond to a post."""
        # Don't respond to our own posts
        if post.get('author') == self.agent_name:
            return False
        
        # Check if it's a question
        text = f"{post.get('title', '')} {post.get('content', '')}"
        analysis = self.detector.analyze(text)
        
        if not analysis['is_question']:
            return False
        
        # Check if we have relevant knowledge
        results = self.knowledge.search(text)
        if not results:
            return False  # Only respond if we have something useful to say
        
        return True
    
    def generate_response(self, post):
        """Generate an educational response."""
        text = f"{post.get('title', '')} {post.get('content', '')}"
        
        # Search knowledge base
        results = self.knowledge.search(text)
        if not results:
            return None
        
        # Format answer
        answer = self.knowledge.format_answer(results, text)
        
        # Log the interaction
        self.teaching_log.append({
            'timestamp': datetime.now().isoformat(),
            'post_id': post.get('id'),
            'question': text[:100],
            'topic': results[0]['topic']
        })
        
        return answer
    
    def reply_to_post(self, post_id, content):
        """Post a reply."""
        response = self.client.create_comment(post_id, content)
        return response.status_code == 200
    
    def scan_and_respond(self, submolts=None, limit=10):
        """
        Scan for questions and respond.
        
        Returns number of responses made.
        """
        if submolts is None:
            submolts = ['general', 'askagents', 'programming']
        
        responses_made = 0
        
        for submolt in submolts:
            response = self.client.get_posts(sort="new", limit=limit, submolt=submolt)
            
            if response.status_code != 200:
                print(f"  Could not fetch m/{submolt}: {response.status_code}")
                continue
            
            posts = response.json()
            if isinstance(posts, dict):
                posts = posts.get('posts', [])
            
            for post in posts:
                post_id = post.get('id')
                
                # Skip if already seen
                if post_id in self.seen_posts:
                    continue
                self.seen_posts.add(post_id)
                
                # Check if we should respond
                if not self.should_respond(post):
                    continue
                
                # Generate and post response
                answer = self.generate_response(post)
                if answer:
                    print(f"  Found question: {post.get('title', 'Untitled')[:50]}...")
                    
                    if self.reply_to_post(post_id, answer):
                        print(f"  ✓ Replied!")
                        responses_made += 1
                    else:
                        print(f"  ✗ Failed to reply (rate limit?)")
        
        return responses_made
    
    def get_stats(self):
        """Get teaching statistics."""
        if not self.teaching_log:
            return {'total': 0, 'topics': {}}
        
        topics = defaultdict(int)
        for entry in self.teaching_log:
            topics[entry['topic']] += 1
        
        return {
            'total': len(self.teaching_log),
            'topics': dict(topics),
            'posts_scanned': len(self.seen_posts)
        }


# Create the teacher
if API_KEY:
    # Get our agent name
    me_response = client.get_me()
    if me_response.status_code == 200:
        agent_name = me_response.json().get('name', 'Unknown')
        teacher = TeacherAgent(client, agent_name)
        print(f"✓ Teacher agent ready as {agent_name}")
    else:
        print("✗ Could not get agent info")

### Step 6: Test the Teacher

In [ ]:
# Simulate a question post
test_post = {
    'id': 'test123',
    'title': 'What is machine learning?',
    'content': 'I keep hearing about ML but I don\'t really understand what it means. Can someone explain?',
    'author': 'CuriousAgent',
    'submolt': 'askagents'
}

print("Test Post:")
print(f"  Title: {test_post['title']}")
print(f"  Author: {test_post['author']}")
print(f"")

# Check if teacher would respond
should_respond = teacher.should_respond(test_post)
print(f"Should respond: {should_respond}")

if should_respond:
    response = teacher.generate_response(test_post)
    print(f"\nGenerated response:\n")
    print(response)

### Step 7: Run a Teaching Session

This scans Moltbook for questions and responds:

In [ ]:
def run_teaching_session(teacher, submolts=None, iterations=1, delay=60):
    """
    Run the teacher agent.
    
    Args:
        teacher: TeacherAgent instance
        submolts: List of submolts to monitor
        iterations: Number of scan cycles
        delay: Seconds between cycles
    """
    if submolts is None:
        submolts = ['general', 'askagents']
    
    print(f"Starting teaching session")
    print(f"  Monitoring: {', '.join(f'm/{s}' for s in submolts)}")
    print(f"  Iterations: {iterations}")
    print(f"  Delay: {delay}s between scans")
    print()
    
    total_responses = 0
    
    for i in range(iterations):
        print(f"--- Scan {i+1}/{iterations} ---")
        
        responses = teacher.scan_and_respond(submolts=submolts, limit=10)
        total_responses += responses
        
        print(f"  Responses this scan: {responses}")
        
        if i < iterations - 1:
            print(f"  Waiting {delay}s...")
            time.sleep(delay)
    
    # Summary
    stats = teacher.get_stats()
    print(f"\n=== Session Complete ===")
    print(f"Total responses: {total_responses}")
    print(f"Posts scanned: {stats['posts_scanned']}")
    print(f"Topics covered: {stats['topics']}")


# Run a single scan (uncomment to execute)
# run_teaching_session(teacher, iterations=1, delay=30)

### Step 8: Add More Knowledge

In [ ]:
# Extend the knowledge base
teacher.knowledge.knowledge['api'] = {
    'summary': 'An Application Programming Interface - a way for programs to communicate with each other.',
    'analogy': 'Like a waiter in a restaurant - you tell them what you want, they bring it from the kitchen.',
    'key_points': [
        'Defines how to request and receive data',
        'Hides complexity behind simple commands',
        'Enables software integration'
    ]
}

teacher.knowledge.knowledge['docker'] = {
    'summary': 'A platform for running applications in isolated containers - like lightweight virtual machines.',
    'analogy': 'Like shipping containers for software - package it once, run it anywhere.',
    'key_points': [
        'Isolates applications from each other',
        'Consistent environment across machines',
        'Critical for security when running AI agents'
    ]
}

teacher.knowledge.knowledge['karma'] = {
    'summary': 'A reputation score on Moltbook based on upvotes and downvotes on your posts and comments.',
    'analogy': 'Like a trust score - helpful contributions earn points, unhelpful ones lose points.',
    'key_points': [
        'Earned through upvoted content',
        'Indicates community standing',
        'Some features may require minimum karma'
    ]
}

print(f"Knowledge base now has {len(teacher.knowledge.knowledge)} topics:")
for topic in sorted(teacher.knowledge.knowledge.keys()):
    print(f"  • {topic}")

## Going Further: RAG Integration

The `KnowledgeBase` above is a simple keyword matcher. For a truly knowledgeable teacher, you need **Retrieval-Augmented Generation (RAG)**.

RAG enables:
- **Semantic search** - Find relevant info even without exact keyword matches
- **Large knowledge bases** - Scale to thousands of topics
- **Dynamic updates** - Add knowledge without code changes
- **Source citations** - Tell students where info comes from

### Recommended: RAG Techniques Repository

**[github.com/NirDiamant/RAG_Techniques](https://github.com/NirDiamant/RAG_Techniques)**

Key techniques for teacher agents:

| Technique | Use Case |
|-----------|----------|
| Basic RAG | Start here - simple vector search |
| Hypothetical Questions | Better retrieval for Q&A |
| Re-ranking | Sort results by relevance |
| Query Expansion | Understand what student really wants |

## Deploying as an OpenClaw Skill

To make your teacher run autonomously, you can package it as an OpenClaw skill.

Create `~/.openclaw/skills/teacher/SKILL.md`:

```markdown
# Teacher Skill

## Trigger
Every 4 hours, or when explicitly requested.

## Actions
1. Scan m/askagents for unanswered questions
2. Check if question matches knowledge base
3. If match found, post helpful response
4. Log teaching interactions

## Constraints
- Only respond to questions we can actually answer
- Respect rate limits (max 50 comments/hour)
- Never claim expertise we don't have
- Be helpful, not preachy
```

OpenClaw will then execute this skill periodically.

## Try It Yourself

1. **Run a teaching session** - Uncomment `run_teaching_session()`
2. **Add topics** - Expand the knowledge base with subjects you know
3. **Monitor performance** - Check which topics get questions
4. **Improve detection** - Add more question patterns
5. **Integrate RAG** - Connect to a vector database for serious knowledge

## What You've Learned

Congratulations! You've completed all 5 tutorials. You now know how to:

1. **Understand Moltbook** - The platform, the hype, the risks
2. **Set up securely** - Docker isolation with proper hardening
3. **Create agents** - Register and interact via the real API
4. **Design personalities** - SOUL.md and behavioral guidelines
5. **Build useful agents** - Question detection, knowledge bases, teaching loops

## Where to Go Next

- **[RAG Techniques](https://github.com/NirDiamant/RAG_Techniques)** - Advanced knowledge retrieval
- **[GenAI Agents](https://github.com/NirDiamant/GenAI_Agents)** - More agent patterns
- **[OpenClaw Docs](https://docs.openclaw.ai)** - Official documentation
- **[DiamantAI Newsletter](https://diamantai.substack.com/)** - Weekly GenAI tutorials

Happy teaching!